In [1]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('imod_g2_141024.csv')

# Display the first few rows of the dataframe
df.head()





,a10,id_lote_g6,g7_1,g10_1,g10_2,g10_3,g10_4,g10_5,g10_c_1,g10_c_2,...,cantidad_kilos_ha_4,cantidad_kilos_ha_5,ax_2,a11,a12,a12a,a13,f4_1,f5_1,_merge
0,DS4001,1,1,triple 18,NaN,NaN,NaN,NaN,Triple 18,NaN,...,NaN,NaN,la andina,Tolima,ESPINAL,NaN,73268001,Fedearroz 67,1.5,Matched (3)
1,DS4001,1,2,triple 18,NaN,NaN,NaN,NaN,Triple 18,NaN,...,NaN,NaN,la andina,Tolima,ESPINAL,NaN,73268001,Fedearroz 67,1.5,Matched (3)
2,DS4001,1,3,triple 18,NaN,NaN,NaN,NaN,Triple 18,NaN,...,NaN,NaN,la andina,Tolima,ESPINAL,NaN,73268001,Fedearroz 67,1.5,Matched (3)
3,DS4001,1,4,triple 18,NaN,NaN,NaN,NaN,Triple 18,NaN,...,NaN,NaN,la andina,Tolima,ESPINAL,NaN,73268001,Fedearroz 67,1.5,Matched (3)
4,DS4001,1,5,triple 18,NaN,NaN,NaN,NaN,Triple 18,NaN,...,NaN,NaN,la andina,Tolima,ESPINAL,NaN,73268001,Fedearroz 67,1.5,Matched (3)


In [2]:
# Read the CSV file
df = pd.read_csv('imod_g2_141024.csv')

# Display the first few rows of the dataframe
df.head()

# Display rows where 'a10' is equal to 'DS4003'
df_ds4003 = df.loc[df['a10'] == 'DS4003']

# Select columns that start with 'Tipo_Nivel_' and include 'g10_1'
df_ds4003_tipo_nivel_1 = df_ds4003[['g10_1','g10_2']].join(df_ds4003.filter(regex='^Tipo_Nivel_'))

# Display the result
df_ds4003_tipo_nivel_1


,g10_1,g10_2,Tipo_Nivel_0_1,Tipo_Nivel_0_2,Tipo_Nivel_0_3,Tipo_Nivel_0_4,Tipo_Nivel_0_5,Tipo_Nivel_1_1,Tipo_Nivel_1_2,Tipo_Nivel_1_3,Tipo_Nivel_1_4,Tipo_Nivel_1_5,Tipo_Nivel_2_1,Tipo_Nivel_2_2,Tipo_Nivel_2_3,Tipo_Nivel_2_4,Tipo_Nivel_2_5
7,triple 18,NaN,Fertilizante edafico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN
8,sulfato de amonio,tripke 18,NaN,Fertilizante edafico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN
9,face 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,amidas,cloruro de potasio,Fertilizante edafico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN
11,nitrabon,NaN,Fertilizante edafico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN
12,azufre boro zinc,NaN,Fertilizante foliar,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN,Fertilizante quimico,NaN,NaN,NaN,NaN


In [3]:
# Perform the reshape to long format for insumos and their compositions

df_new2_long = pd.wide_to_long(df,
                               stubnames=['g10_c', 'K2O', 'Ntotal', 'Namoniacal', 'Nnitrico', 'Nureico', 'P2O5',
                                          'cantidad_litros_ha', 'cantidad_kilos_ha','Tipo_Nivel_0','Tipo_Nivel_1'], 
                               i=['a10', 'g7_1','a12'], 
                               j='insumo_num', 
                               sep='_', 
                               suffix='\\d+').reset_index()

# Dropping rows where there are no insumos
df_new2_long_clean = df_new2_long.dropna(subset=['g10_c'])
df_new2_long_clean.head()


# Save the reshaped dataframe to a CSV file
df_new2_long_clean.to_csv('modulo_g2_reshape.csv', index=False)



In [4]:
# Obtener los nombres únicos de la columna 'g10_c'
nombres_g10_c = df_new2_long_clean['a12'].unique()

# Ordenar los nombres de mayor a menor
nombres_g10_c_ordenados = sorted(nombres_g10_c, reverse=True)

nombres_g10_c_ordenados

['VENADILLO',
 'SALDAÑA',
 'PURIFICACIÓN',
 'PRADO',
 'PIEDRAS',
 'LÉRIDA',
 'IBAGUÉ',
 'HERVEO',
 'GUAMO',
 'FLANDES',
 'ESPINAL',
 'COYAIMA',
 'ARMERO',
 'AMBALEMA',
 'ALVARADO']

In [5]:
# Grouping by 'a10' (hogar) and 'g10_c' (insumo), summing the quantities, and keeping the first composition values for each insumo
df_insumo_sum_compositions_full = df_new2_long_clean.groupby(['a10', 'g10_c']).agg(
    municipio=('a12', 'first'),
    total_cantidad_litros_ha=('cantidad_litros_ha', 'sum'),
    total_cantidad_kilos_ha=('cantidad_kilos_ha', 'sum'),
    K2O=('K2O', 'first'),
    Ntotal=('Ntotal', 'first'),
    Namoniacal=('Namoniacal', 'first'),
    Nnitrico=('Nnitrico', 'first'),
    Nureico=('Nureico', 'first'),
    P2O5=('P2O5', 'first'),
    Tipo_Nivel_0=('Tipo_Nivel_0', 'first'),
    Tipo_Nivel_1=('Tipo_Nivel_1', 'first')
).reset_index()



# Save the reshaped dataframe to a CSV file
df_insumo_sum_compositions_full.to_csv('modulo_g2_reshape_grouped.csv', index=False)

df_insumo_sum_compositions_full 

,a10,g10_c,municipio,total_cantidad_litros_ha,total_cantidad_kilos_ha,K2O,Ntotal,Namoniacal,Nnitrico,Nureico,P2O5,Tipo_Nivel_0,Tipo_Nivel_1
0,DS4001,Triple 18,ESPINAL,0.0,1000.0,18.0,18.00,7.00,0.00,11.00,18.0,Fertilizante edafico,Fertilizante quimico
1,DS4001,Urea,ESPINAL,0.0,100.0,0.0,46.00,0.00,0.00,46.00,0.0,Fertilizante edafico,Fertilizante quimico
2,DS4003,(Nutrimon) Ecofertil fase 3,ESPINAL,0.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,DS4003,Borozinco,ESPINAL,1.0,0.0,0.0,3.45,0.83,2.28,0.35,0.0,Fertilizante foliar,Fertilizante quimico
4,DS4003,KCl,ESPINAL,0.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719,RS3143,KCl,LÉRIDA,0.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1720,RS3143,Sulfato de Amonio,LÉRIDA,0.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1721,RS3143,Sulfato de amonio SAM,LÉRIDA,0.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1722,RS3143,Triple 18,LÉRIDA,0.0,150.0,18.0,18.00,7.00,0.00,11.00,18.0,Fertilizante edafico,Fertilizante quimico


In [6]:


df_insumo_sum_compositions_full.head()
# Multiplicar total_cantidad_litros_ha por las variables K2O, Ntotal, Namoniacal, Nnitrico, Nureico, P2O5 y dividir por 100
df_insumo_sum_compositions_full['litros_K2O'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['K2O']) / 100
df_insumo_sum_compositions_full['litros_Ntotal'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['Ntotal']) / 100
df_insumo_sum_compositions_full['litros_Namoniacal'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['Namoniacal']) / 100
df_insumo_sum_compositions_full['litros_Nnitrico'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['Nnitrico']) / 100
df_insumo_sum_compositions_full['litros_Nureico'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['Nureico']) / 100
df_insumo_sum_compositions_full['litros_P2O5'] = (df_insumo_sum_compositions_full['total_cantidad_litros_ha'] * df_insumo_sum_compositions_full['P2O5']) / 100

# Multiplicar total_cantidad_kilos_ha por las variables K2O, Ntotal, Namoniacal, Nnitrico, Nureico, P2O5 y dividir por 100
df_insumo_sum_compositions_full['kilos_K2O'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['K2O']) / 100
df_insumo_sum_compositions_full['kilos_Ntotal'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['Ntotal']) / 100
df_insumo_sum_compositions_full['kilos_Namoniacal'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['Namoniacal']) / 100
df_insumo_sum_compositions_full['kilos_Nnitrico'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['Nnitrico']) / 100
df_insumo_sum_compositions_full['kilos_Nureico'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['Nureico']) / 100
df_insumo_sum_compositions_full['kilos_P2O5'] = (df_insumo_sum_compositions_full['total_cantidad_kilos_ha'] * df_insumo_sum_compositions_full['P2O5']) / 100

df_insumo_sum_compositions_full.head()

# Save the reshaped dataframe to a CSV file
df_insumo_sum_compositions_full.to_csv('modulo_g2_reshape_grouped_calculated.csv', index=False)
df_insumo_sum_compositions_full.head()


,a10,g10_c,municipio,total_cantidad_litros_ha,total_cantidad_kilos_ha,K2O,Ntotal,Namoniacal,Nnitrico,Nureico,...,litros_Namoniacal,litros_Nnitrico,litros_Nureico,litros_P2O5,kilos_K2O,kilos_Ntotal,kilos_Namoniacal,kilos_Nnitrico,kilos_Nureico,kilos_P2O5
0,DS4001,Triple 18,ESPINAL,0.0,1000.0,18.0,18.00,7.00,0.00,11.00,...,0.0000,0.0000,0.0000,0.0,180.0,180.0,70.0,0.0,110.0,180.0
1,DS4001,Urea,ESPINAL,0.0,100.0,0.0,46.00,0.00,0.00,46.00,...,0.0000,0.0000,0.0000,0.0,0.0,46.0,0.0,0.0,46.0,0.0
2,DS4003,(Nutrimon) Ecofertil fase 3,ESPINAL,0.0,150.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DS4003,Borozinco,ESPINAL,1.0,0.0,0.0,3.45,0.83,2.28,0.35,...,0.0083,0.0228,0.0035,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DS4003,KCl,ESPINAL,0.0,50.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Display the data types of the columns in df_insumo_sum_compositions_full
df_insumo_sum_compositions_full.dtypes

a10                          object
g10_c                        object
municipio                    object
total_cantidad_litros_ha    float64
total_cantidad_kilos_ha     float64
K2O                         float64
Ntotal                      float64
Namoniacal                  float64
Nnitrico                    float64
Nureico                     float64
P2O5                        float64
Tipo_Nivel_0                 object
Tipo_Nivel_1                 object
litros_K2O                  float64
litros_Ntotal               float64
litros_Namoniacal           float64
litros_Nnitrico             float64
litros_Nureico              float64
litros_P2O5                 float64
kilos_K2O                   float64
kilos_Ntotal                float64
kilos_Namoniacal            float64
kilos_Nnitrico              float64
kilos_Nureico               float64
kilos_P2O5                  float64
dtype: object

In [8]:


# Agrupar por g10_c y calcular promedios de las columnas especificadas
df_promedios_por_g10_c = df_insumo_sum_compositions_full.groupby('g10_c').agg({
    'Tipo_Nivel_0': 'first',
    'Tipo_Nivel_1': 'first',
    'total_cantidad_litros_ha': 'mean',
    'total_cantidad_kilos_ha': 'mean',
    'litros_K2O': 'mean',
    'litros_Ntotal': 'mean',
    'litros_Namoniacal': 'mean',
    'litros_Nnitrico': 'mean',
    'litros_Nureico': 'mean',
    'litros_P2O5': 'mean',
    'kilos_K2O': 'mean',
    'kilos_Ntotal': 'mean',
    'kilos_Namoniacal': 'mean',
    'kilos_Nnitrico': 'mean',
    'kilos_Nureico': 'mean',
    'kilos_P2O5': 'mean'
}).reset_index()


# Renombrar las columnas agregadas para agregar el prefijo 'av_'
df_promedios_por_g10_c = df_promedios_por_g10_c.rename(columns={
    'total_cantidad_litros_ha': 'av_total_cantidad_litros_ha',
    'total_cantidad_kilos_ha': 'av_total_cantidad_kilos_ha',
    'litros_K2O': 'av_litros_K2O',
    'litros_Ntotal': 'av_litros_Ntotal',
    'litros_Namoniacal': 'av_litros_Namoniacal',
    'litros_Nnitrico': 'av_litros_Nnitrico',
    'litros_Nureico': 'av_litros_Nureico',
    'litros_P2O5': 'av_litros_P2O5',
    'kilos_K2O': 'av_kilos_K2O',
    'kilos_Ntotal': 'av_kilos_Ntotal',
    'kilos_Namoniacal': 'av_kilos_Namoniacal',
    'kilos_Nnitrico': 'av_kilos_Nnitrico',
    'kilos_Nureico': 'av_kilos_Nureico',
    'kilos_P2O5': 'av_kilos_P2O5'
})


# Mostrar el resultado
df_promedios_por_g10_c.head()
# Save the reshaped dataframe to a CSV file
df_promedios_por_g10_c.to_csv('modulo_g2_reshape_grouped_calculated_promedios.csv', index=False)

In [9]:
# Definir las columnas que queremos agregar
columns_to_agg = {
    'Tipo_Nivel_0': 'first',
    'Tipo_Nivel_1': 'first',
    'total_cantidad_litros_ha': 'mean',
    'total_cantidad_kilos_ha': 'mean',
    'litros_K2O': 'mean',
    'litros_Ntotal': 'mean',
    'litros_Namoniacal': 'mean',
    'litros_Nnitrico': 'mean',
    'litros_Nureico': 'mean',
    'litros_P2O5': 'mean',
    'kilos_K2O': 'mean',
    'kilos_Ntotal': 'mean',
    'kilos_Namoniacal': 'mean',
    'kilos_Nnitrico': 'mean',
    'kilos_Nureico': 'mean',
    'kilos_P2O5': 'mean'
}
# Agrupar por municipio y calcular promedios de las columnas especificadas
df_promedios_por_municipio = df_insumo_sum_compositions_full.groupby('municipio').agg(columns_to_agg).reset_index()
# Renombrar las columnas agregadas para agregar el prefijo 'av_'
df_promedios_por_municipio = df_promedios_por_municipio.rename(columns={
    'total_cantidad_litros_ha': 'av_total_cantidad_litros_ha',
    'total_cantidad_kilos_ha': 'av_total_cantidad_kilos_ha',
    'litros_K2O': 'av_litros_K2O',
    'litros_Ntotal': 'av_litros_Ntotal',
    'litros_Namoniacal': 'av_litros_Namoniacal',
    'litros_Nnitrico': 'av_litros_Nnitrico',
    'litros_Nureico': 'av_litros_Nureico',
    'litros_P2O5': 'av_litros_P2O5',
    'kilos_K2O': 'av_kilos_K2O',
    'kilos_Ntotal': 'av_kilos_Ntotal',
    'kilos_Namoniacal': 'av_kilos_Namoniacal',
    'kilos_Nnitrico': 'av_kilos_Nnitrico',
    'kilos_Nureico': 'av_kilos_Nureico',
    'kilos_P2O5': 'av_kilos_P2O5'
})



# Mostrar el resultado
df_promedios_por_municipio.head()

# Save the reshaped dataframe to a CSV file
df_promedios_por_municipio.to_csv('modulo_g2_reshape_grouped_calculated_promedios_municipio.csv', index=False)

In [10]:
import pandas as pd

# Definir las columnas que queremos agregar
columns_to_agg = {
    'Tipo_Nivel_0': 'first',
    'Tipo_Nivel_1': 'first',
    'total_cantidad_litros_ha': 'mean',
    'total_cantidad_kilos_ha': 'mean',
    'litros_K2O': 'mean',
    'litros_Ntotal': 'mean',
    'litros_Namoniacal': 'mean',
    'litros_Nnitrico': 'mean',
    'litros_Nureico': 'mean',
    'litros_P2O5': 'mean',
    'kilos_K2O': 'mean',
    'kilos_Ntotal': 'mean',
    'kilos_Namoniacal': 'mean',
    'kilos_Nnitrico': 'mean',
    'kilos_Nureico': 'mean',
    'kilos_P2O5': 'mean'
}

# Agrupar por municipio y luego por g10_c, y calcular promedios de las columnas especificadas
df_promedios_por_municipio_g10_c = df_insumo_sum_compositions_full.groupby(['municipio', 'g10_c']).agg(columns_to_agg).reset_index()

# Renombrar las columnas agregadas para agregar el prefijo 'av_'
df_promedios_por_municipio_g10_c = df_promedios_por_municipio_g10_c.rename(columns={
    'total_cantidad_litros_ha': 'av_total_cantidad_litros_ha',
    'total_cantidad_kilos_ha': 'av_total_cantidad_kilos_ha',
    'litros_K2O': 'av_litros_K2O',
    'litros_Ntotal': 'av_litros_Ntotal',
    'litros_Namoniacal': 'av_litros_Namoniacal',
    'litros_Nnitrico': 'av_litros_Nnitrico',
    'litros_Nureico': 'av_litros_Nureico',
    'litros_P2O5': 'av_litros_P2O5',
    'kilos_K2O': 'av_kilos_K2O',
    'kilos_Ntotal': 'av_kilos_Ntotal',
    'kilos_Namoniacal': 'av_kilos_Namoniacal',
    'kilos_Nnitrico': 'av_kilos_Nnitrico',
    'kilos_Nureico': 'av_kilos_Nureico',
    'kilos_P2O5': 'av_kilos_P2O5'
})
# Mostrar el resultado
df_promedios_por_municipio_g10_c.head()
#save the reshaped dataframe to a CSV file
df_promedios_por_municipio_g10_c.to_csv('modulo_g2_reshape_grouped_calculated_promedios_municipio_g10_c.csv', index=False)